In [ ]:
import numpy as np
from my_stuff import BinanceUSDMTestKeys, ApexTestKeys, MufexTestKeys, BybitTestKeys,MufexKeys
from quantfreedom.helper_funcs import candles_to_df
from quantfreedom.exchanges.binance_exchange.binance_usdm import BinanceUSDM
from quantfreedom.exchanges.bybit_exchange.bybit import Bybit
from quantfreedom.enums import *
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.exchanges.apex_exchange.apex import Apex
from quantfreedom.simulate import run_df_backtest, or_backtest
from quantfreedom.strategies.strategy import Strategy

%load_ext autoreload
%autoreload 2

In [ ]:
mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

%load_ext autoreload
%autoreload 2

In [ ]:
candles = mufex_main.get_candles(
    symbol="BTCUSDT",
    timeframe="5m",
    candles_to_dl=550,
    since_date_ms=1699988400000,
    until_date_ms=1700092800000,
)

candles_to_df(candles=candles)

In [ ]:
mufex_main.set_exchange_settings(
    symbol="BTCUSDT",
    position_mode=PositionModeType.HedgeMode,
    leverage_mode=LeverageModeType.Isolated,
)

# backtest_settings = BacktestSettings(
#     array_size=200,
#     gains_pct_filter=-90,
#     total_trade_filter=10,
#     upside_filter=-np.inf,
# )
backtest_settings = BacktestSettings()
dos_arrays = DynamicOrderSettingsArrays(
    max_equity_risk_pct=np.array([12]),
    max_trades=np.array([5]),
    risk_account_pct_size=np.array([3]),
    risk_reward=np.array([2.5]),
    sl_based_on_add_pct=np.array([0.05, 0.1, 0.15]),
    sl_based_on_lookback=np.array([20]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([0.5]),
    trail_sl_when_pct=np.array([1]),
)
static_os = StaticOrderSettings(
    starting_bar=50,
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    pg_min_max_sl_bcb="min",
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    sl_to_be_bool=False,
    starting_equity=1000.0,
    static_leverage=None,
    tp_fee_type="limit",
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    trail_sl_bool=True,
    z_or_e_type=None,
)
strategy = Strategy(
    rsi_is_below=np.array([60]),
    rsi_length=np.array([14]),
    long_short="long",
)

In [ ]:
run_df_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=mufex_main.exchange_settings,
    static_os=static_os,
    strategy=strategy,
    logger_bool=False,
)

In [ ]:
order_records_df = or_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=mufex_main.exchange_settings,
    static_os=static_os,
    strategy=strategy,
    dos_index=2,
    ind_set_index=0,
    plot_results=True,
    logger_bool=True,
)

In [ ]:
order_records_df

In [ ]:
Mufex(
    api_key=MufexTestKeys.api_key,
    secret_key=MufexTestKeys.secret_key,
    use_test_net=True,
).create_order(
    symbol="BTCUSDT",
    position_mode=PositionModeType.BuySide,
    buy_sell="buy",
    order_type="market",
    asset_size=0.001,
    time_in_force="GoodTillCancel",
)

In [ ]:
Apex(
    api_key=ApexTestKeys.api_key,
    secret_key=ApexTestKeys.secret_key,
    passphrase=ApexTestKeys.passphrase,
    stark_key_private=ApexTestKeys.stark_key_private,
    stark_key_public=ApexTestKeys.stark_key_public,
    stark_key_y=ApexTestKeys.stark_key_y,
    use_test_net=True,
).create_entry_market_order(
    symbol="BTC-USDC",
    buy_sell="sell",
    asset_size=0.005,
)

In [ ]:
BinanceUSDM(
    api_key=BinanceUSDMTestKeys.api_key,
    secret_key=BinanceUSDMTestKeys.secret_key,
    use_test_net=True,
).create_order(
    symbol="BTCUSDT",
    side="BUY",
    type="MARKET",
    quantity=0.005,
    timeInForce=None,
)

In [ ]:
Bybit(
    use_test_net=True,
    api_key=BybitTestKeys.api_key,
    secret_key=BybitTestKeys.secret_key,
).create_order(
    symbol="BTCUSDT",
    buy_sell="buy",
    position_mode=1,
    order_type="market",
    asset_size=0.01,
)